In [ ]:
%matplotlib inline

import os
import cv2
import math
import imutils
import contextlib2
import pandas as pd
import tensorflow as tf

from matplotlib import pyplot as plt
from concurrent.futures import ThreadPoolExecutor

pd.set_option("display.max_colwidth", 10000)

In [ ]:
ROOT_OUT = ""
SPLIT = "validation" # ["train", "test", "validation"]
FILTERED_URL_CSV = f"{SPLIT}/{SPLIT}-filtered-img-url-v2.csv"
OUTDIR = os.path.join(ROOT_OUT, SPLIT)

In [ ]:
urls = pd.read_csv(FILTERED_URL_CSV)

In [ ]:
def download_and_resize(url, file_id, output_dir=""):
    try:
        img = imutils.url_to_image(url)
        img = imutils.resize(img, width=640, height=480)
        height, width, channels = img.shape

        if output_dir and not os.path.exists(output_dir):
            os.mkdir(output_dir)

        file_path = os.path.join(output_dir, f"{file_id}.jpg")
        cv2.imwrite(file_path, img)
                
        return file_id, file_path, None, width, height, channels
    except Exception as ex:
        return file_id, None, ex, -1, -1, -1

In [ ]:
channel_problems=[]
with ThreadPoolExecutor() as executor:
    results = executor.map(lambda x: download_and_resize(x[1].image_url, x[1].ImageID, output_dir=OUTDIR), urls.head(100).iterrows())

    for fileId, filePath, ex, width, height, channels in results:
        if ex:
            print(f"Download of image {fileId} failed with error '{ex}'")
        else:
            print(f"Image {fileId} with shape ({width}, {height}, {channels}) downloaded to '{filePath}'")
            if channels != 3:
                channel_problems.append(fileId)
                print(f"======================== Imagem {fileId} com canais diferentes de 3! ========================")

In [ ]:
len(channel_problems)